In [2]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cac


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    separators=[r'(?<=[.!?])\s+', "\n"],
)

loader = Docx2txtLoader('./data/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

print(len(document_list))
print(f'각 청크 길이 {list(len(text.page_content) for text in document_list)}')

225
각 청크 길이 [1466, 1421, 1427, 1396, 1398, 1481, 1251, 1483, 1471, 1480, 1479, 1425, 1494, 1458, 1492, 1482, 1493, 1429, 1363, 1437, 1463, 1450, 1481, 1437, 1441, 1456, 1433, 1319, 1458, 1496, 1461, 1382, 1384, 1479, 1494, 1435, 1376, 1479, 1470, 1481, 1333, 1412, 1477, 1014, 1456, 1362, 1446, 1386, 1055, 1467, 1361, 1493, 1473, 1493, 1428, 1443, 1470, 1486, 1290, 1442, 1421, 1496, 1478, 1288, 1492, 1458, 1441, 1237, 1476, 1416, 1431, 1429, 1445, 1460, 1483, 1399, 1484, 1481, 1333, 1308, 1385, 1479, 1495, 1386, 1375, 1345, 1353, 1382, 1446, 1356, 1409, 1441, 1443, 1431, 1494, 1226, 1413, 1468, 1396, 1455, 1488, 1496, 1446, 1353, 1397, 1427, 1443, 1440, 1478, 1443, 1436, 1458, 1360, 1434, 1429, 1443, 1412, 1494, 1414, 1478, 1471, 1373, 1492, 1429, 1493, 1377, 1438, 1389, 1305, 1446, 1160, 1403, 1465, 1371, 1342, 1427, 1235, 1363, 1460, 1387, 1277, 1473, 1386, 1410, 1460, 1457, 1489, 1488, 1335, 1454, 1411, 1488, 1482, 1448, 1265, 1412, 1352, 1377, 1459, 1451, 1288, 1450, 1481, 1459, 147

In [ ]:
# 임베딩
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
# from langchain_chroma import Chroma

# # 데이터 처음 저장할 때
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name='chroma-tax',
#     persist_directory='./chroma'
# )

In [6]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pincone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pincone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [49]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
retriever.invoke(query)

[Document(id='32bb69bc-b478-4a68-aa5c-d7182f449106', metadata={'source': './data/tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제

In [50]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [51]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=['context', 'question'],
#     template=""" 
# [Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해 주세요.  

# [Context]
# {context}

# Question: {question}
# """
# )

In [52]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

In [53]:
ai_message = qa_chain.invoke({"query": query})

In [54]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '연봉 5천만 원인 직장인의 소득세는 근로소득에 적용되는 기본세율에 따라 결정됩니다. 기본세율은 누진세율로, 세율과 소득구간에 따라 달라집니다. 연봉 및 공제 사항에 따라 달라질 수 있으니, 정확한 세액을 계산하기 위해서는 구체적인 세율표 및 공제 항목을 참고하는 것이 필요합니다.'}

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}

    질문: {{question}}

""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [ ]:
# new_question = dictionary_chain.invoke({"question": query})
# new_question

'질문: 연봉 5천만원 거주자 소득세는 얼마인가?'

In [ ]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인의 소득세는 얼마인가?"

In [66]:
ai_message = tax_chain.invoke({"question": query})

In [67]:
ai_message

{'query': '연봉 5천만원 거주자의 소득세는 얼마인가?',
 'result': '연봉 5천만원 거주자의 소득세는 624만원입니다. 이는 과세표준 5천만원을 초과하지 않으므로 해당 구간의 세율을 적용하여 계산한 금액입니다.'}